In [3]:
import os

with open(f'test.tex') as f:
        reader = f.read()

reader_size = len(reader)

break_phrase = '\\rule[0.5ex]{1\columnwidth}{1pt}'

found_index = reader.find(break_phrase)

qns = []

while 0 <= found_index < reader_size:
    next_index = reader.find(break_phrase, found_index + len(break_phrase))
    qns.append(reader[found_index + len(break_phrase):next_index])
    found_index = next_index
    if found_index == -1:
        break

qns = [qn.strip()[len('\\begin{enumerate}'):-
                len('\\begin{enumerate}')+1].strip()+'\n' for qn in qns[:-1]]

for q in qns:
     print(q)
     exam_type, year, paper_no, question_no = q[q.find('{[}')+len('{[}'):q.find('{]}')].split('/')
     print(exam_type, year, paper_no, question_no)

\item \textbf{{[}ACJC/PROMO/9758/2021/Q1{]} }

State a sequence of transformations that will transform the curve
with equation $y=2\sin(2x+\alpha)\cos(x)$ on to the curve with equation
$y=-2\sin(4x+3\alpha)\cos(2x+\alpha$ ), where $\alpha$ is a positive
constant. \hfill{}{[}3{]}



ValueError: too many values to unpack (expected 4)